### Creating and Persisting an ML Model

In [468]:
import pandas as pd
import numpy as np
df = pd.read_csv('data/student-mat.csv', sep=';')

Summary of the data

In [469]:
df.describe()

,age,Medu,Fedu,traveltime,studytime,failures,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
count,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000
mean,16.696203,2.749367,2.521519,1.448101,2.035443,0.334177,3.944304,3.235443,3.108861,1.481013,2.291139,3.554430,5.708861,10.908861,10.713924,10.415190
std,1.276043,1.094735,1.088201,0.697505,0.839240,0.743651,0.896659,0.998862,1.113278,0.890741,1.287897,1.390303,8.003096,3.319195,3.761505,4.581443
min,15.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,3.000000,0.000000,0.000000
25%,16.000000,2.000000,2.000000,1.000000,1.000000,0.000000,4.000000,3.000000,2.000000,1.000000,1.000000,3.000000,0.000000,8.000000,9.000000,8.000000
50%,17.000000,3.000000,2.000000,1.000000,2.000000,0.000000,4.000000,3.000000,3.000000,1.000000,2.000000,4.000000,4.000000,11.000000,11.000000,11.000000
75%,18.000000,4.000000,3.000000,2.000000,2.000000,0.000000,5.000000,4.000000,4.000000,2.000000,3.000000,5.000000,8.000000,13.000000,13.000000,14.000000
max,22.000000,4.000000,4.000000,4.000000,4.000000,3.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,75.000000,19.000000,19.000000,20.000000


In [470]:
df.info

<bound method DataFrame.info of     school sex  age address famsize Pstatus  Medu  Fedu      Mjob      Fjob  \
0       GP   F   18       U     GT3       A     4     4   at_home   teacher   
1       GP   F   17       U     GT3       T     1     1   at_home     other   
2       GP   F   15       U     LE3       T     1     1   at_home     other   
3       GP   F   15       U     GT3       T     4     2    health  services   
4       GP   F   16       U     GT3       T     3     3     other     other   
..     ...  ..  ...     ...     ...     ...   ...   ...       ...       ...   
390     MS   M   20       U     LE3       A     2     2  services  services   
391     MS   M   17       U     LE3       T     3     1  services  services   
392     MS   M   21       R     GT3       T     1     1     other     other   
393     MS   M   18       R     LE3       T     3     2  services     other   
394     MS   M   19       U     LE3       T     1     1     other   at_home   

     ... famrel fre

Create a subset of features as an example.

In [471]:
include = ['health', 'age', 'absences', 'famsup', 'internet', 'reason', 'studytime', 'G3']
df['famsup'] = np.where(df['famsup']=='yes', 1, 0)
df['internet'] = np.where(df['internet']=='yes', 1, 0)
def quantifyreason(reason):
    if reason == 'home':
        return 1;
    elif reason == 'reputation':
        return 2;
    elif reason == 'course':
        return 3;
    elif reason == 'other':
        return 0;
df['reason'] = df['reason'].apply(quantifyreason)
df.drop(columns=df.columns.difference(include), inplace=True)

In [472]:
df.info

<bound method DataFrame.info of      age  reason  studytime  famsup  internet  health  absences  G3
0     18       3          2       0         0       3         6   6
1     17       3          2       1         1       3         4   6
2     15       0          2       0         1       3        10  10
3     15       1          3       1         1       5         2  15
4     16       1          2       1         0       5         4  10
..   ...     ...        ...     ...       ...     ...       ...  ..
390   20       3          2       1         0       4        11   9
391   17       3          1       0         1       2         3  16
392   21       3          1       0         0       3         3   7
393   18       3          1       0         1       5         0  10
394   19       3          1       0         1       5         5   9

[395 rows x 8 columns]>

The goal is to predict the quality of the student. We will build a predictor based on the final grade (G3).
Becasue we are trying to find quality students. In this model we define a quality student as one who achieves a final grade of 15 or higher. 

In [473]:
df['qual_student'] = np.where(df['G3']>=15, 1, 0)

In [474]:
df.describe()

,age,reason,studytime,famsup,internet,health,absences,G3,qual_student
count,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000
mean,16.696203,1.908861,2.035443,0.612658,0.832911,3.554430,5.708861,10.415190,0.184810
std,1.276043,1.000912,0.839240,0.487761,0.373528,1.390303,8.003096,4.581443,0.388636
min,15.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
25%,16.000000,1.000000,1.000000,0.000000,1.000000,3.000000,0.000000,8.000000,0.000000
50%,17.000000,2.000000,2.000000,1.000000,1.000000,4.000000,4.000000,11.000000,0.000000
75%,18.000000,3.000000,2.000000,1.000000,1.000000,5.000000,8.000000,14.000000,0.000000
max,22.000000,3.000000,4.000000,1.000000,1.000000,5.000000,75.000000,20.000000,1.000000


Drop the G3 score

In [475]:
include = ['absences', 'health', 'age', 'famsup', 'internet', 'reason', 'studytime', 'qual_student']
df.drop(columns=df.columns.difference(include), inplace=True) 

Import scikit-learn and build a random forest classifer

In [476]:
from sklearn.ensemble import RandomForestClassifier as rf
import sklearn
from sklearn.model_selection import train_test_split

dependent_variable = 'qual_student'
x = df[df.columns.difference([dependent_variable])]
y = df[dependent_variable]

X_train, X_test, y_train,y_test= train_test_split(x, y, test_size=0.3)

clf = rf(n_estimators = 1000)
clf.fit(X_train, y_train)

RandomForestClassifier(n_estimators=1000)

In [477]:
from sklearn.metrics import accuracy_score, log_loss

pred = clf.predict(x)
print(sklearn.metrics.f1_score(y, pred, average='binary'))
print('****Random Forest Classifier Results****')
train_predictions = clf.predict(X_test)
acc = accuracy_score(y_test, train_predictions)
print("Accuracy: {:.4%}".format(acc))

0.7343750000000001
****Random Forest Classifier Results****
Accuracy: 73.1092%


In [478]:
from sklearn import tree
clftree = tree.DecisionTreeClassifier(max_depth=3)
clftree.fit(X_train, y_train)
# Finding the accuracy of decision tree

from sklearn.metrics import accuracy_score, log_loss

print('****Decision Tree Classifier Results****')
train_predictions = clftree.predict(X_test)
acc = accuracy_score(y_test, train_predictions)
print("Accuracy: {:.4%}".format(acc))

****Decision Tree Classifier Results****
Accuracy: 72.2689%


In [479]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(X_train, y_train)

print('****Logistic Regression Results****')
train_predictions = lr.predict(X_test)
acc = accuracy_score(y_test, train_predictions)
print("Accuracy: {:.4%}".format(acc))

****Logistic Regression Results****
Accuracy: 76.4706%


In [480]:
from sklearn.svm import SVC

svc = SVC()
svc.fit(X_train, y_train)

print('****SVC Results****')
train_predictions = svc.predict(X_test)
acc = accuracy_score(y_test, train_predictions)
print("Accuracy: {:.4%}".format(acc))

****SVC Results****
Accuracy: 76.4706%


In [481]:
from sklearn.ensemble import GradientBoostingClassifier

gbc = GradientBoostingClassifier()
gbc.fit(X_train, y_train)

print('****GBC Results****')
train_predictions = gbc.predict(X_test)
acc = accuracy_score(y_test, train_predictions)
print("Accuracy: {:.4%}".format(acc))

****GBC Results****
Accuracy: 69.7479%


In [482]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
knn.fit(X_train, y_train)

print('****KNN Results****')
train_predictions = knn.predict(X_test)
acc = accuracy_score(y_test, train_predictions)
print("Accuracy: {:.4%}".format(acc))

****KNN Results****
Accuracy: 73.1092%


In [483]:
from sklearn.linear_model import SGDClassifier

sgd = SGDClassifier()
sgd.fit(X_train, y_train)

print('****SGD Results****')
train_predictions = sgd.predict(X_test)
acc = accuracy_score(y_test, train_predictions)
print("Accuracy: {:.4%}".format(acc))

****SGD Results****
Accuracy: 76.4706%


It's not very good! We didn't even cross validate. You'll need to do better :)
Let's export this model so we can use it in a microservice (flask api)

In [341]:
import joblib
# modify the file path to where you want to save the model
joblib.dump(clf, 'app/handlers/model.pkl')

['app/handlers/model.pkl']

In [375]:
query = [[0, 5, 20, 1, 1, 1, 3]]
print(clf.predict_proba(query))
print('goat' if clf.predict(query) == 1 else 'clown')

[[0.37426667 0.62573333]]
goat


/usr/local/lib/python3.10/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [376]:
x

,absences,age,famsup,health,internet,reason,studytime
0,6,18,0,3,0,3,2
1,4,17,1,3,1,3,2
2,10,15,0,3,1,0,2
3,2,15,1,5,1,1,3
4,4,16,1,5,0,1,2
...,...,...,...,...,...,...,...
390,11,20,1,4,0,3,2
391,3,17,0,2,1,3,1
392,3,21,0,3,0,3,1
393,0,18,0,5,1,3,1


In [354]:
type(x)

pandas.core.frame.DataFrame